In [110]:
import pandas as pd
import numpy as np
from datetime import datetime

In [173]:
stock = pd.read_csv(r'"C:\Users\bangy\Desktop\1.csv"')
stock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 485 entries, 0 to 484
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       485 non-null    object 
 1   Open       485 non-null    float64
 2   High       485 non-null    float64
 3   Low        485 non-null    float64
 4   Close      485 non-null    float64
 5   Adj Close  485 non-null    float64
 6   Volume     485 non-null    int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 26.6+ KB


In [174]:
open_price = stock["Open"]
close_price = stock["Close"]
high_price =stock["High"]
low_price = stock["Low"]
close_price = stock["Close"]

In [175]:
high_channel = high_price.rolling(window = 20).max()
low_channel = low_price.rolling(window = 20).min()

In [179]:
true_range = []
previous_close = close_price[0]
for high, low, close in zip(high_price, low_price, close_price):
    current_range = max(high - low, abs(high - previous_close), abs(previous_close - low))
    true_range.append(current_range)
    previous_close = close

true_range = pd.Series(true_range)
atr = true_range.rolling(window=20).mean() - true_range.rolling(window=1).mean() + true_range
print(atr)

0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
         ...    
480    50.083997
481    50.099500
482    50.481995
483    49.646997
484    50.034497
Length: 485, dtype: float64


In [180]:
total_money = 1000000
current_money = total_money
total_unit = 0
entry_price = 0
for days in range(19, len(atr)):
    unit = int(current_money * 0.01 / (100 * atr[days]))

    if entry_price == 0 and high_price[days] >= high_channel[days]:
        unit = min(unit, int(current_money/(100 * high_price[days])))
        current_money = current_money - unit * high_price[days] * 100
        total_unit = total_unit + unit
        entry_price = high_price[days]
        continue

    if high_price[days] - 0.5 * atr[days] >= entry_price:
        unit = min(unit, int(current_money/(100 * high_price[days])))
        current_money = current_money - unit * high_price[days] * 100
        total_unit = total_unit + unit
        continue

    if total_unit > 0 and low_price[days] <= low_channel[days]:
        current_money = current_money + total_unit * low_price[days] * 100
        total_unit = 0
        entry_price = 0
        continue

    if total_unit > 0 and low_price[days] + 2 * atr[days] <= entry_price:
        unit = min(total_unit, unit)
        current_money = current_money + unit * low_price[days] * 100
        total_unit = total_unit - unit
        if(total_unit == 0):
            entry_price = 0
        continue

final_money = current_money + total_unit * 100 * close_price[len(close_price)-1]
print(final_money)

868151.0253000001


In [181]:
start_date = datetime(2010, 6, 29)
end_date = datetime(2017, 3, 17)

num_days = (end_date - start_date).days

print("Number of days:", num_days)

Number of days: 2453


In [182]:
stock = int(total_money / 100 / open_price[0])
print("Origin: Annualized rate of return:", (stock * close_price[len(close_price)-1] * 100/ total_money - 1) / num_days * 365)

Origin: Annualized rate of return: -0.09010366431981248


In [183]:
print("Strategy: Rate of return:", (final_money / total_money - 1))
print("Strategy: Annualized rate of return:", (final_money / total_money - 1) / num_days * 365)

Strategy: Rate of return: -0.1318489746999999
Strategy: Annualized rate of return: -0.01961878343477373
